In [4]:
## Can also get the price chart by pokemon number, or url for past prices. Look intro trading strategies; also, work on real-time offering feed vs history. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import re

# Customize number of pages to scrape
num_pages = 1
base_url = 'https://www.tcgplayer.com/search/pokemon/product?productLineName=pokemon&view=grid&page={}'

# Setup headless Chrome options
options = Options()
options.add_argument("--headless=new")
prefs = {"profile.default_content_setting_values": {"images": 2, "stylesheets": 2}}
options.add_experimental_option("prefs", prefs)


def scrape_page(page_num):
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    url = base_url.format(page_num)
    driver.get(url)

    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-card__product")))
    except:
        driver.quit()
        return []

    products = driver.find_elements(By.CLASS_NAME, "product-card__product")
    results = []

    for product in products:
        # --- Get name and product ID ---
        try:
            img_elem = product.find_element(By.TAG_NAME, "img")
            name = img_elem.get_attribute("alt").strip()

            # Try src first, then srcset if missing
            src = img_elem.get_attribute("src")
            if not src or "product/" not in src:
                srcset = img_elem.get_attribute("srcset")
                if srcset:
                    src = srcset.split(" ")[0]  # first URL in srcset

            # Extract product ID (the 6+ digit number)
            id_match = re.search(r"/product/(\d+)", src)
            product_id = id_match.group(1) if id_match else None

            # Build product detail link
            product_link = f"https://www.tcgplayer.com/product/{product_id}" if product_id else None
        except Exception:
            name = None
            product_link = None

        # --- Set Name ---
        try:
            set_name = product.find_element(By.CLASS_NAME, "product-card__set-name__variant").text.strip()
        except Exception:
            set_name = None

        # --- Market Price (→ float) ---
        try:
            mktprice_text = product.find_element(By.CLASS_NAME, "product-card__market-price--value").text.strip()
            mktprice_match = re.search(r"\$([\d,.]+)", mktprice_text)
            mktprice = float(mktprice_match.group(1).replace(",", "")) if mktprice_match else None
        except Exception:
            mktprice = None

        # --- Listings count (→ int) ---
        try:
            listings_span = product.find_element(By.CLASS_NAME, "inventory__listing-count").text.strip()
            listings_match = re.search(r"(\d+)\s+listings", listings_span)
            listings = int(listings_match.group(1)) if listings_match else None
        except Exception:
            listings = None

        # --- Append Row ---
        results.append({
            "name": name,
            "link": product_link,   # now always a clean product page link
            "set": set_name,
            "mktprice": mktprice,   # numeric float
            "listings": listings    # numeric int
        })

    driver.quit()
    return results


# --- Concurrent scraping across pages ---
all_results = []
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(scrape_page, p) for p in range(1, num_pages + 1)]
    for future in as_completed(futures):
        all_results.extend(future.result())

# --- Save results ---
pricedf = pd.DataFrame(all_results)

# Print DataFrame
print(pricedf)


                                             name  link  \
0        Code Card - Destined Rivals Booster Pack  None   
1   Code Card - Prismatic Evolutions Booster Pack  None   
2            Code Card - White Flare Booster Pack  None   
3             Code Card - Black Bolt Booster Pack  None   
4                                           Hilda  None   
5                                         Pikachu  None   
6                                     Air Balloon  None   
7                                 Arven - 166/198  None   
8                                   Victini - 208  None   
9                                  Iono - 080/091  None   
10                                   Prism Energy  None   
11                                Night Stretcher  None   
12                                   Brave Bangle  None   
13                                Luminous Energy  None   
14                        Black Bolt Booster Pack  None   
15                             Buddy-Buddy Poffin  None 

In [6]:
pricedf

,name,link,set,mktprice,listings
0,Code Card - Destined Rivals Booster Pack,None,SV10: Destined Rivals,$0.05,223
1,Code Card - Prismatic Evolutions Booster Pack,None,SV: Prismatic Evolutions,$0.04,222
2,Code Card - White Flare Booster Pack,None,SV: White Flare,$0.08,96
3,Code Card - Black Bolt Booster Pack,None,SV: Black Bolt,$0.06,99
4,Hilda,None,SV: White Flare,$4.22,170
5,Pikachu,None,Celebrations,$7.23,287
6,Air Balloon,None,SV: Black Bolt,$1.55,275
7,Arven - 166/198,None,SV01: Scarlet & Violet Base Set,$2.51,555
8,Victini - 208,None,SV: Scarlet & Violet Promo Cards,$7.38,182
9,Iono - 080/091,None,SV: Paldean Fates,$0.29,942
